In [1]:
# pip install requests BeautifulSoup4 lxml
# pip install pygame
import requests
from bs4 import BeautifulSoup
import pandas as pd
import math
#from tqdm import tqdm
import time
#import os,  glob
#import csv
import psycopg2
from sqlalchemy import create_engine

# Офрмление
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

# Для подключение к БД                                  
HOST    ='HOST'
DB      ='DB'
DBUSER  ='DBUSER'
DBPASS  ='DBPASS'
PORT    ='PORT'

greenplum = psycopg2.connect(host = HOST, database = DB, user = DBUSER, password = DBPASS)
gp_cursor = greenplum.cursor()

OperationalError: could not translate host name "HOST" to address: Unknown host


In [ ]:
# Оформление
%%html
<style>
.cell-output-ipywidget-background {
   background-color: transparent !important;
}
.jp-OutputArea-output {
   background-color: transparent;
}  
</style>

### Создаем класс 

In [ ]:
class Response:
    def __init__(self):
        self.url = 'https://www.auchan.ru'
        self.start_time = time.time()
        self.style= {'bar_color':'#f08080'}
        self.layout = Layout(width='20%', height='20px')

    def start_respond(self, type):
        if type == 'test':
            response = requests.get(self.url, verify=False)
            response.raise_for_status()
            print(response)
        
        elif type == 'main':
            main_links = []
            folder_name = []
            response = requests.get(self.url, verify=False)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'lxml')
            
            for link in soup.find_all('___', '________________'):
                main_links.append(self.url+link.get('href'))
                folder_name.append(link.get('href').replace('/catalog/', "").replace('/', "").replace('-', "_"))
    
            return main_links
        
    def respond(self, links, type, page_type):
        progress = IntProgress(min = 0, max = len(links), style=self.style, layout=self.layout)
        display(progress)
        num = 0
        resp_links = []

        for s_link in links:
            try:
                response = requests.get(s_link, verify=False)
                response.raise_for_status()
                soup = BeautifulSoup(response.text, 'lxml')
        
                if type == 'category':

                    for link in soup.find_all('___', '________________'):
                        resp_links.append(self.url+link.get('href'))
        
                elif type == 'uncategory':

                    for link in soup.find_all('___', '________________'):
                        resp_links.append(self.url+link.get('href'))

                elif type == 'pages':

                    if page_type == 'category':
                        n = soup.find('___', '________________').text            # Для количества товаров в категории
                    elif page_type == 'uncategory':
                        n = soup.find('___', '________________').text            # Для количества товаров в подкатегории
                        
                    page = 1
                    n = n.replace('(',"").replace(')', "")
                    max_page = math.ceil(int(n)/40)

                    while page <= max_page:
                        resp_links.append(s_link+'?page={}'.format(page))
                        page += 1

                elif type == 'products':

                    for link in soup.find_all('___', '________________'):
                        resp_links.append(s_link+link.get('href'))

                num += 1    
                perc = round((num / len(links)) * 100, 2)
                end_time = time.time()
                predict = round((((end_time - self.start_time)/num)*(len(links)-num))/60,2)
                print (' PROCESS: {}/{}  ||  {} %  ||  WAITING TIME {} MIN  ||   '.format(num, len(links), perc, predict), end='\r')
                
                output = wi.Output()
                progress.value = num
                    
            except Exception as e:
                print ('SOMETHING IS WRONG WITH {}      '.format(s_link), end='\r')
                pass
            
        return resp_links

def redactor(links, type, want_to_split):           # TESTING
    
    prod = pd.DataFrame(links)
    prod.to_csv ("full_path.csv", index=False)

    if type == 'category':
        names=['1','2','3','4','category','uncategory','5','product','name','6']
    elif type == 'uncategory':
        names=['1','2','3','4','category','uncategory','subcategory','5','product','name','6']
    
    path_prod=pd.read_csv('full_path.csv', names=names, encoding='cp1251', sep='/', skiprows=1)
    path_prod = path_prod.drop(columns=['1','2','3','4','5','6'])
    path_prod['path'] = 'https://www.auchan.ru/product/' + path_prod['name'] +'/'

    if want_to_split == True:
        
    return path_prod

R = response()

In [ ]:
links = R.start_respond(type = 'main')

In [ ]:
cat_links = R.respond(type = 'category', links = links)

### Сохраняем полную информацию о ссылках на продукты

df_full_path = pd.DataFrame(full_path_art)
df_full_path.to_csv ("full_path.csv", index=False)

In [ ]:
path_prod=pd.read_csv('full_path.csv',names=['1','2','3','4','category','subcategory','5','product','name','6'],
encoding='cp1251',sep='/', skiprows=1)
path_prod = path_prod.drop(columns=['1','2','3','4','5','6'])
path_prod['path'] = 'https://www.auchan.ru/product/' + path_prod['name'] +'/'
path_prod.head()

# Преобразуем данные для разбиения

In [ ]:
path_prod=pd.read_csv('full_path.csv',names=['1','2','3','4','category','subcategory','5','product','name','6'],
encoding='cp1251',sep='/', skiprows=1)
path_prod = path_prod.drop(columns=['1','2','3','4','5','6'])
path_prod['path'] = 'https://www.auchan.ru/product/' + path_prod['name'] +'/'

cat = path_prod.category.unique()
files_name = []

for cat_name in cat:
    df = path_prod.loc[(path_prod['category'] == cat_name)]
    df = df.drop(columns=['category','subcategory','product','name'])
    file_name = "{}_part".format(cat_name)
    df.to_csv ('{}.csv'.format(file_name), index= False, header= False, sep=';')
    files_name.append(file_name)

prodycts_links = []
for i in files_name:
    prodycts_links.append(i+'.csv')

## Выбор категории для загрузки товаров

print(prodycts_links)

In [ ]:
file  = 'aziya_part.csv'        # выбрать категорию из списка выше
prod_cat=pd.read_csv(file, names=['links'])
prod_cat = prod_cat['links'].to_list()

# Собираем имя товара и артикул

In [ ]:
start_time = time.time()
name_art = []
links = 0
leng = len(prod_cat)
progress = IntProgress(min = 0, max = leng, value = links, style={'bar_color': '#f08080'}, layout=Layout(width='20%', height='20px'))
display(progress)      

for file in prod_cat:
    try:
        response = requests.get(file, verify=False)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'lxml')

        name_art.append(file + soup.find('__','___________').text + '/'+ soup.find('td','css-1v23ygr').text)

        links = links + 1
        perc = round((links / leng) * 100, 2)
        end_time = time.time()
        predict = round((((end_time - start_time)/links)*(leng-links))/60,2)
        print (' PROCESS: {}/{}  ||  {} %  ||  WAITING TIME {} MIN    '.format(links,leng,perc,predict), end='\r')

    except Exception as e:
        print (' SOMETHING IS WRONG WITH {}           '.format(url), end='\r')
        pass

    output = wi.Output()
    progress.value = links

#print(folder_name)

In [ ]:
df = pd.DataFrame(name_art)
df.to_csv ("df_final.csv", index=False)       # УКАЗАТЬ ИМЯ ДЛЯ ИТОГОВОГО ФАЙЛА

df = pd.read_csv('df_final.csv')
df = df['0'].str.split('/', expand=True)
df.columns=['0','1','2','3','Имя_ENG','Наименование','Артикул','7']
df = df.drop(columns=['0','1','2','3','7'])
df.dtypes

## Объединение полученных данных с данными из Dbeaver

In [ ]:
query_prod = """
select sku, lvl_1, lvl_2, lvl_3, lvl_4 from products
"""

In [ ]:
products  =  pd.read_sql(query_prod, greenplum)
products['sku'] = products['sku'].astype(str)
products.dtypes

In [ ]:
df_merged = pd.merge(products, df, left_on='sku', right_on='Артикул')
df_merged = df_merged[['Артикул', 'Наименование', 'lvl_1', 'lvl_2', 'lvl_3','lvl_4']]
df_merged = df_merged.drop_duplicates ()
df_merged

### Запись артикулов и продуктов и загрузка в SAS

In [ ]:
df_art = pd.DataFrame(name_art)
df_art.to_csv ("prd_art.csv", index=False)

In [ ]:
# Для загрузки в базу
conn = create_engine('________________________________________________')

table_path = 'table_table'          #УКАЗАТЬ НАИМЕНОВАНИЕ ТАБЛИЦЫ В SAS
df_merged.to_sql(name=table_path, con=conn, schema='public', if_exists='replace')
print('Залито в базу, путь: {}'.format(table_path))